# Tutorial data

In [ ]:
# import pandas as pd

# data_path = "data/train/day_0_0.001_train.csv"

# df = pd.read_csv(data_path, header=None)
# df.head()

In [ ]:
# example = df.loc[1, 0]
# example

In [ ]:
# features = example.split('\t')
# len(features)
# features

# Read s3 orc data

In [2]:
column_names = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        column_names.append(line.split(' ')[1].strip())
print(column_names)

['_11001', '_11002', '_11003', '_11004', '_11007', '_11008', '_11021', '_11022', '_11023', '_11024', '_11041', '_11042', '_11043', '_11044', '_11045', '_11046', '_11061', '_11062', '_11063', '_11064', '_11065', '_11066', '_11081', '_11082', '_11083', '_11084', '_11085', '_11086', '_11601', '_11602', '_11603', '_12001', '_12002', '_12003', '_12004', '_12005', '_12006', '_20001', '_20002', '_20003', '_20101', '_20102', '_20201', '_20202', '_20203', '_20204', '_20205', '_20206', '_20207', '_20208', '_20209', '_20210', '_30001', '_30002', '_30003', '_30004', '_30005', '_30006', '_30201', '_30202', '_30203', '_30204', '_30205', '_30206', '_30207', '_40001', '_40002', '_40003', '_40004', '_40005', '_40201', '_40202', '_40203', '_40204', '_40205', '_40206', '_40207', '_40208', '_40209', '_40210', '_40211', '_40212', '_40213', '_40214', '_40215', '_40231', '_40301', '_40302', '_40303', '_40304', '_40305', '_40306', '_40307', '_40321', '_40322', '_40323', '_40324', '_50801', '_50802', '_50805',

In [5]:
import metaspore as ms

# train_dataset_path = ROOT_DIR + '/data/train/day_0_0.001_train.csv'

file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 200
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

# train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/part-00000-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc'
train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
train_dataset_path = [
    's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/',
    's3://mv-mtg-di-for-poc-datalab/2024/06/14/01/',
    's3://mv-mtg-di-for-poc-datalab/2024/06/14/02/',
]


spark_confs = {
    'spark.eventLog.enabled':'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=2,
                                     server_count=2,
                                     log_level='WARN',
                                     spark_confs=spark_confs)


train_dataset = ms.input.read_s3_csv(spark_session, 
                                    train_dataset_path, 
                                    format='orc',
                                    shuffle=False,
                                    delimiter='\t', 
                                    multivalue_delimiter="\001", 
                                    column_names=column_names,
                                    multivalue_column_names=column_names[:-1])

# train_dataset.printSchema()
train_dataset.count()

ignore shuffle


In [3]:
from tqdm import tqdm
import pandas as pd
pd.set_option("display.max_columns", None)

columns = column_names[:-1]
feature_hash = {}
feature_value = {}

num_files_per_chunk = 10
num_chunks = num_files // num_files_per_chunk

num_rows = 0

for i in tqdm(range(num_chunks), total=num_chunks):
    chunk_path = train_dataset_path[i*num_files_per_chunk:(i+1)*num_files_per_chunk]

    # Read the chunk
    train_dataset = ms.input.read_s3_csv(spark_session, 
                                     chunk_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=column_names,
                                     multivalue_column_names=column_names[:-1])
    
    # Convert the chunk to a Pandas DataFrame
    df = train_dataset.toPandas()
    # df.head()
    num_rows += df.shape[0]

    # num_positive = df[df['label']=='1'].shape[0]
    # num_negative = df[df['label']=='0'].shape[0]
    # print(f"num_positive: {num_positive}, num_negative: {num_negative}, ratio: {num_positive/num_negative*100}")

    for i, row in df.iterrows(): # tqdm(df.iterrows(), total=df.shape[0]):
        for column in columns[:-1]:
            if column in feature_value:
                feature_value[column].append(row[column].shape[0])
            else:
                feature_value[column] = [row[column].shape[0]]
            
            for item in row[column]:
                if '\003' in item:
                    hash, weight = item.split('\003')
                else:
                    hash, weight = '', 0
                if column in feature_hash:
                    feature_hash[column].add(hash)
                else:
                    feature_hash[column] = set(hash)

print(f"Number of files: {num_files}, number of rows: {num_rows}")

  0%|          | 0/20 [00:00<?, ?it/s]

ignore shuffle


2024-07-21 13:06:21,635 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
  5%|▌         | 1/20 [04:17<1:21:35, 257.67s/it]                               

ignore shuffle


2024-07-21 13:10:49,547 ERROR executor.Executor: Exception in task 2.0 in stage 1.0 (TID 16)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3745)
	at java.base/java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:120)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:95)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:156)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1873)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1782)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1180)
	at java.base/java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:345)
	at org.apache.spark.serializer.JavaSerializationStream.writeObject(JavaSerializer.scala:44

Py4JJavaError: An error occurred while calling o48801.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:97)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:93)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands

2024-07-21 13:10:50,049 WARN datasources.FileScanRDD: Skipped the rest of the content in the corrupted file: path: s3a://mv-mtg-di-for-poc-datalab/2024/06/14/00/part-00017-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc, range: 0-134217728, partition values: [empty row]
java.lang.IllegalStateException: Connection pool shut down
	at com.amazonaws.thirdparty.apache.http.util.Asserts.check(Asserts.java:34)
	at com.amazonaws.thirdparty.apache.http.impl.conn.PoolingHttpClientConnectionManager.requestConnection(PoolingHttpClientConnectionManager.java:269)
	at jdk.internal.reflect.GeneratedMethodAccessor27.invoke(Unknown Source)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at com.amazonaws.http.conn.ClientConnectionManagerFactory$Handler.invoke(ClientConnectionManagerFactory.java:76)
	at com.amazonaws.http.conn.$Proxy18.requestConnection(Unknown Source)
	at com.

In [ ]:
df_feature = pd.DataFrame(feature_value)
df_feature.describe()

In [ ]:
feature_hash_count = dict([(k, [len(v)]) for k, v in feature_hash.items()])
df_hash = pd.DataFrame(feature_hash_count)
df_hash

In [ ]:
hashes = set()

for k, v in feature_hash.items():
    hashes |= v
print(f"Number of total hashes: {len(hashes)}")

In [ ]:
total_hash = df_hash.sum(axis=1).values[0]
print(f"Number of total hashes: {total_hash}")

# Use pyspark to analyze

In [ ]:
columns = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        columns.append(line.split(' ')[1].strip())
print(columns)

In [ ]:
import metaspore as ms
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, explode, sum as spark_sum, countDistinct
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType

# Read column names from file
columns = []
with open('./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        columns.append(line.split(' ')[1].strip())
print(columns)

# Define Spark configuration and session
file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 1
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

spark_confs = {
    'spark.eventLog.enabled': 'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=3,
                                     server_count=3,
                                     log_level='WARN',
                                     spark_confs=spark_confs)

# Load the dataset
train_dataset = ms.input.read_s3_csv(spark_session, 
                                     train_dataset_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=columns,
                                     multivalue_column_names=columns[:-1])

print(f"Number of orcs: {num_files}, total number of rows: {train_dataset.count()}")

# Define UDF to process rows
def process_row(column_values):
    feature_hashes = set()
    for value in column_values:
        if '\003' in value:
            hash_val, weight = value.split('\003')
        else:
            hash_val, weight = '', 0
        feature_hashes.add(hash_val)

    feature_values = len(column_values)
    return (feature_values, list(feature_hashes))

# Define schema for UDF output
schema = StructType([
    StructField("feature_values", IntegerType(), True),
    StructField("hashes", ArrayType(StringType()), True)
])

# Register the UDF
process_row_udf = udf(process_row, schema)

# Apply UDF on each column and process the dataset
for column in columns[:-1]:
    train_dataset = train_dataset.withColumn(f'{column}_processed', process_row_udf(col(column)))

train_dataset.show()


In [ ]:
# Extract the processed columns
result_columns = [f'{column}_processed' for column in columns[:-1]]
processed_df = train_dataset.select(*result_columns)
processed_df.show()

In [ ]:
# Select the desired row and column
row_value = train_dataset.select('_11002_processed').collect()[4][0]

# Print the value
print(row_value)

In [ ]:
# 1. Obtain the percentiles of feature_values using approxQuantile
from tqdm import tqdm
percentiles = [0.25, 0.5, 0.75, 1.0]

for col_name in tqdm(processed_df.columns, total=len(processed_df.columns)):
    feature_values_df = processed_df.select(col(f"{col_name}.feature_values").cast("double").alias("feature_values"))
    
    # Print schema to verify cast
    # feature_values_df.printSchema()
    
    # Show data to verify cast
    # feature_values_df.show()
    
    # Verify the data type of the column
    # data_type = feature_values_df.schema["feature_values"].dataType
    # print(f"Data type of feature_values in {col_name}: {data_type}")
    
    # # Check for null values or any anomalies in the column
    # null_count = feature_values_df.filter(col("feature_values").isNull()).count()
    # print(f"Number of null values in feature_values of {col_name}: {null_count}")
    
    # Obtain approximate percentiles
    try:
        percentile_values = feature_values_df.approxQuantile("feature_values", percentiles, 0.01)
        # print(f"Percentiles for {col_name}: {percentile_values}")
    except Exception as e:
        print(f"Error occurred while calculating percentiles for {col_name}: {e}")

# 2. Obtain the total number of different hash values for each column and across all columns
distinct_hash_counts = {}
all_hashes_df = []

print(f"Starting counting hashes")
for col_name in tqdm(processed_df.columns, total=len(processed_df.columns)):
    hashes_df = processed_df.select(explode(col(f"{col_name}.hashes")).alias("hashes"))
    distinct_count = hashes_df.select(countDistinct("hashes")).first()[0]
    distinct_hash_counts[col_name] = distinct_count
    all_hashes_df.append(hashes_df)

union_hashes_df = all_hashes_df[0]
for df in all_hashes_df[1:]:
    union_hashes_df = union_hashes_df.union(df)

total_distinct_hashes = union_hashes_df.select(countDistinct("hashes")).first()[0]

print(f"Distinct hash counts per column: {distinct_hash_counts}")
print(f"Total distinct hashes across all columns: {total_distinct_hashes}")

# Stop the Spark session
# spark.stop()